# AI Model Tracing

This notebook demonstrates how to use tracing with the Azure AI Inference client library.
It will guide you through setting up tracing, defining functions with tracing, and interacting with an AI model.


### 🔧 Task: Configure Azure AI Inference Client

In [ ]:
import os
import json
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage, UserMessage, CompletionsFinishReason,
    ToolMessage, AssistantMessage, ChatCompletionsToolCall,
    ChatCompletionsToolDefinition, FunctionDefinition
)
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor

load_dotenv('../../.env')

ai_project_endpoint=os.environ["AI_PROJECT_ENDPOINT"]
app_insights_connection_string=os.environ["APP_INSIGHTS_CONNECTION_STRING"]
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"

# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=ai_project_endpoint,
    credential=DefaultAzureCredential()
)

# Enable Azure Monitor tracing
configure_azure_monitor(connection_string=app_insights_connection_string)

## Defining Functions with Tracing

In [ ]:
import requests
from opentelemetry.trace import get_tracer

tracer = get_tracer(__name__)

weather_api_key = os.environ.get("WEATHER_API_KEY")

@tracer.start_as_current_span("get_temperature") 
def get_temperature(city: str) -> str:
    span = trace.get_current_span()
    span.set_attribute("requested_city", city)
    url = f"http://api.weatherapi.com/v1/current.json?key={weather_api_key}&q={city}&aqi=no"
    try: 
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        temp_c = data["current"]["temp_c"]
        return {"temperature_celsius": temp_c}
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

@tracer.start_as_current_span("get_weather")
def get_weather(city: str) -> str:
    span = trace.get_current_span()
    span.set_attribute("requested_city", city)
    url = f"http://api.weatherapi.com/v1/current.json?key={weather_api_key}&q={city}&aqi=no"
    try:
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        condition = data["current"]["condition"]["text"]
        return {"condition": condition}
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

## Chat Completion with Function Calls

In [ ]:
def chat_completion_with_function_call(key, endpoint):
    weather_description = ChatCompletionsToolDefinition(
        function=FunctionDefinition(
            name="get_weather",
            description="Returns description of the weather in the specified city",
            parameters={
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "City name"}
                },
                "required": ["city"],
            },
        )
    )

    temperature_in_city = ChatCompletionsToolDefinition(
        function=FunctionDefinition(
            name="get_temperature",
            description="Returns the current temperature (in Celsius) for the specified city",
            parameters={
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "City name"}
                },
                "required": ["city"],
            },
        )
    )

    client = ChatCompletionsClient(
        endpoint=endpoint, 
        credential=AzureKeyCredential(key),
    )
    messages = [
        SystemMessage(content="""
        You are an AI assistant with access to various tools to enhance your capabilities. 
        Your primary goal is to assist the user efficiently and accurately by leveraging these tools when necessary. 
        Below are the tools you can use:
        - `get_weather`: Returns the description of the weather in the specified city.
        - `get_temperature`: Returns the current temperature for the specified city.
        """),
        UserMessage(content="What is the weather and temperature in Bordeaux?"),
    ]
    response = client.complete(
        messages=messages, 
        tools=[weather_description, temperature_in_city],
    )

    if response.choices[0].finish_reason == CompletionsFinishReason.TOOL_CALLS:
        messages.append(AssistantMessage(tool_calls=response.choices[0].message.tool_calls))
        if response.choices[0].message.tool_calls:
            for tool_call in response.choices[0].message.tool_calls:
                if isinstance(tool_call, ChatCompletionsToolCall):
                    function_args = json.loads(tool_call.function.arguments.replace("'", '"'))
                    print(f"Calling function `{tool_call.function.name}` with arguments {function_args}")
                    function_response = globals()[tool_call.function.name](**function_args)
                    print(f"Function response = {function_response}")
                    messages.append(
                        ToolMessage(
                            content=str(function_response),
                            tool_call_id=tool_call.id)
                        )
            response = client.complete(
                messages=messages, 
                tools=[weather_description, temperature_in_city],
                temperature=0.0,
            )
    print(f"Model response = {response.choices[0].message.content}")

## Running the AI Model and Tracing

In [ ]:
from azure.ai.inference.tracing import AIInferenceInstrumentor

key=os.environ["AZURE_AI_CHAT_KEY"]
endpoint=os.environ["AZURE_AI_CHAT_ENDPOINT"]

AIInferenceInstrumentor().instrument()
try:
    chat_completion_with_function_call(key,endpoint)
finally:
    AIInferenceInstrumentor().uninstrument()